# Intro to HPC: Solar Cell project

Project: Solar Power for Affordable Housing through Computational Design of Low-Cost/High-Efficiency Solar Cells.
Author: AlvaroVM [https://alvarovm.github.io](http://alvarovm.github.io)

Version: 0.0.2

## Part 2 Cleaning and cannonizing DB

For this second part we will need to complete the dataset transformation in part 1 and save the results in a file.

Author: AlvaroVM [https://alvarovm.github.io](http://alvarovm.github.io)
Version: 0.0.1

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
SRC_DIR='..'
sys.path.append(os.path.join(SRC_DIR, 'code'))
import utils

import pandas as pd
#https://github.com/jmcarpenter2/swifter
#import swifter
#2-TSNE-UMAP-map-cuda-Copy1

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs 
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint
from rdkit.DataStructs import ConvertToNumpyArray

from sklearn.manifold import TSNE

import hdbscan

import seaborn as sns

utils.plot_settings2()

results_path = os.path.join(SRC_DIR,'results')

In [ ]:
df = pd.read_pickle('../data/extended_db_Zindo_Nov_2019_V5_cannfp.pkl').fillna(value = 0)
print('Column names: {}'.format(str(df.columns.tolist())))
print('Table Shape: {}'.format(df.shape))


### Exercises
* Use `df.count()` and `df.hist()` to have idea of the dataset distribution

* Find the molecules with the smalles `lambda_sTDA (nm)`, for example those with values  more less than 200 nm. Do they have anything in common?

* Plot a distribution function with `sns.distplot()` for `lambda_exp_min (nm)` and `lambda_exp_max (nm)`


* Compute the difference between `lambda_sTDA (nm)` and `lambda_exp_min (nm)`, and plot the distribution of this difference


* Plot `lambda_exp_min (nm)` vs `lambda_exp_max (nm)`

* Plot `lambda_exp_min (nm)` vs `lambda_exp_max (nm)`


* Plot `lambda_exp_min (nm)` vs `lambda_sTDA (nm)`


* Plot the distribution function of the molecules that absorb light in the UV/Vis spectra, eg. `200 < lambda_sTDA (nm) < 800`


* Compare and plot the diffenrece between `lambda_sTDA (nm)` and `lambda_z (nm)`


* Find the relations using `scatter_matrix` among the excitation energies predicted with the methods `gapdft`, `gapz`, `gapmopac`, `lambda_z (nm)`, `lambda_sTDA (nm)`, `lambda_tddft (nm)`, `lambda_exp_max (nm)`, `lambda_exp_min (nm)`. Which values correlate better?


* Find the relations using `scatter_matrix`  among the  absortion with the methods `f1_sTDA`,`f1_ZINDO`, `f1_TDDFT`, `ε_Exp_max`


* Do a bar plot to visualize the distrution of number of ring (`NumAromaticRings`) using sns.barplot, use `df['NumAromaticRings'].value_counts()`


* Do a bar plot to visualize the distrution of number of ring (`NumAromaticHeterocycles`) using sns.barplot function


* Compare the absorption of `f1_sTDA`  and `NumAromaticRings`


* Do a scatter plot that compares 'gapdft' with `lambda_tddft (nm)` and color the points with `NumAromaticRings`


* Find the systems with more than 10 aromatic rings. Do they have anything in common? Do they absorb more light or have a darker color?


* Find those systems with more than 10 aromatic rings and have values different to zero in `lambda_z (nm)` and `lambda_exp_min (nm)`, do those values correlate?